## **Serialbox Tutorial : Incorporating Fortran Serialbox Data into Python**

In the [previous notebook](./01.ipynb), we covered how to extract data from a Fortran code using Serialbox.  In this notebook, we'll cover how to read and incorporate those files within a Python code.

### **Notebook Requirements**

- Python v3.11.x to v3.12.x
- [NOAA/NASA Domain Specific Language Middleware](https://github.com/NOAA-GFDL/NDSL)
- `ipykernel==6.1.0`
- [`ipython_genutils`](https://pypi.org/project/ipython_genutils/)

This notebook assumes that the code from the [previous notebook](./01.ipynb) was run, and the serialized data from Fortran was written out.

### **Importing Fortran Serialbox Data From Example 1 into Python** ###

We'll step through importing Serialbox data [created in Fortran](./01.ipynb#Serialbox-Example-1) into Python to test a Python port of `FILLQ2ZERO1`.  Importing Serialbox data into Python essentially comes from opening a file via a "serializer" object denoted by a particular Serialbox initialization prefix (see [Serialbox directive calls in Fortran code](./01.ipynb#Serialbox-directive-calls-in-Fortran-code)) and stepping through the savepoints within the "serializer" object to read the data.  This is done by the following Python calls assuming that the imported `serialbox` package is referenced via `ser`.

- `ser.Serializer(ser.OpenModeKind.Read,"<Path to Serialbox Data>", "<Name of prefix used during Serialbox initialization>")` : This function call creates a "serializer" object that will read Serialbox files within a declared path and reference data from a particular Serialbox initialization prefix.

- `serializer.savepoint_list()` : Using a "serializer" object called `serializer`, this function call creates a list of Serialbox savepoints

- `serializer.read("<Serialbox variable name>", <Savepoint from savepoint list>)` : Using a "serializer" object called `serializer`, this function call will look for the specified Serialbox variable name from the savepoint list and output that variable.

Below is a Python example that uses these three calls to import the [Example 1](./01.ipynb#Serialbox-Example-1) Fortran data into Python.  You can check to see that the summation of the arrays with Python match closely with the [values presented in Fortran](./01.ipynb#Building-and-Running-Fortran-code-with-Serialbox-library).

In [1]:
import sys
# Appends the Serialbox python path to PYTHONPATH.  If needed, change to appropriate path containing serialbox installation
sys.path.append('/home/ckung/Documents/Code/SMT-Nebulae/sw_stack_path/install/serialbox/python')
import serialbox as ser
import numpy as np

# If needed, change the path in second parameter of ser.Serializer to appropriate path that contains Fortran data via Serialbox from 01.ipynb
serializer = ser.Serializer(ser.OpenModeKind.Read,"./Fortran/sb/","FILLQ2ZERO_InOut")

savepoints = serializer.savepoint_list()

Qin_out = serializer.read("q_in", savepoints[0])
mass    = serializer.read("m_in", savepoints[0])
fq_out  = serializer.read("fq_in", savepoints[0])

print('Sum of Qin_out = ', sum(sum(sum(Qin_out))))
print('Sum of mass = ', sum(sum(sum(mass))))
print('Sum of fq_out = ', sum(sum(fq_out)))

Sum of Qin_out =  58.74463748931885
Sum of mass =  62.169867515563965
Sum of fq_out =  0.0


Next, we'll create a rudimentary port of `fillq2zero1` and test whether or not it computes properly by comparing the output arrays `Qin_out` and `fq_out` to the corresonding arrays created from Fortran, which are retrieved using `serializer.read()`.  In this example, the comparison between the Fortran and Python data is performed using `np.allclose`; however, note that the proper metric of comparison will depend on the application.  We'll see that `np.allclose()` will report `True` for both the `Qin_out` and `fq_out` array comparisons. 

In [2]:
def fillq2zero1(Q, MASS, FILLQ):
    IM = Q.shape[0]
    JM = Q.shape[1]
    LM = Q.shape[2]

    TPW = np.sum(Q*MASS,2)
    for J in range(JM):
        for I in range(IM):
            NEGTPW = 0.
            for L in range(LM):
                if(Q[I,J,L] < 0.0):
                    NEGTPW = NEGTPW + (Q[I,J,L]*MASS[I,J,L])
                    Q[I,J,L] = 0.0
            for L in range(LM):
                if(Q[I,J,L] >= 0.0):
                    Q[I,J,L] = Q[I,J,L]*(1.0 + NEGTPW/(TPW[I,J]-NEGTPW))
            FILLQ[I,J] = -NEGTPW
            
fillq2zero1(Qin_out,mass,fq_out)

print('Sum of Qin_out = ', sum(sum(sum(Qin_out))))
print('Sum of fq_out = ', sum(sum(fq_out)))

Qin_out_ref = serializer.read("q_out", savepoints[0])
mass_ref    = serializer.read("m_out", savepoints[0])
fq_out_ref  = serializer.read("fq_out", savepoints[0])

print(np.allclose(Qin_out,Qin_out_ref))
print(np.allclose(fq_out,fq_out_ref))

Sum of Qin_out =  58.78519535064697
Sum of fq_out =  0.25218469463288784
True
True


### **Importing Fortran Data from Example 2 into Python : Looping Regions** ###

In [Example 2](./01.ipynb#Serialbox-Example-2), Serialbox was set up to record data within a looping region.  This results in a larger list of savepoints that we can step through in Python to recreating the looping process done in Fortran.  The code below replicates the looping of `FILLQ2ZERO1` and reads multiple savepoints to intialize the data and compare outputs.

In [3]:
# If needed, change the path in second parameter of ser.Serializer to appropriate path that contains Fortran data via Serialbox from 01.ipynb
serializer = ser.Serializer(ser.OpenModeKind.Read,"./Fortran_ts/sb/","FILLQ2ZERO_InOut")

savepoints = serializer.savepoint_list()

for currentSavepoint in savepoints:
    Qin_out = serializer.read("q_in", currentSavepoint)
    mass    = serializer.read("m_in", currentSavepoint)
    fq_out  = serializer.read("fq_in", currentSavepoint)

    fillq2zero1(Qin_out,mass,fq_out)

    Qin_out_ref = serializer.read("q_out", currentSavepoint)
    mass_ref    = serializer.read("m_out", currentSavepoint)
    fq_out_ref  = serializer.read("fq_out", currentSavepoint)

    print('Current savepoint = ', currentSavepoint)
    print(np.allclose(Qin_out,Qin_out_ref))
    print(np.allclose(fq_out,fq_out_ref))

Current savepoint =  sp1 {"timestep": 1}
True
True
Current savepoint =  sp1 {"timestep": 2}
True
True
Current savepoint =  sp1 {"timestep": 3}
True
True
Current savepoint =  sp1 {"timestep": 4}
True
True
Current savepoint =  sp1 {"timestep": 5}
True
True
Current savepoint =  sp1 {"timestep": 6}
True
True
Current savepoint =  sp1 {"timestep": 7}
True
True
Current savepoint =  sp1 {"timestep": 8}
True
True
Current savepoint =  sp1 {"timestep": 9}
True
True
Current savepoint =  sp1 {"timestep": 10}
True
True
